In [1]:
import numpy as np
import pandas as pd
import os
import time
import pandas_gbq
from google.cloud import bigquery


In [2]:
os.chdir('G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Dados para subir\\Tabelas finais')


In [3]:
df = pd.read_csv('SIAPE_remuneracao_liquida_executivo_raca_escolaridade.csv')
df

,ano,escolaridade,raca,liquido
0,1999,1º grau completo ou menos,Amarela,"3320,7"
1,1999,1º grau completo ou menos,Branca,"3543,9"
2,1999,1º grau completo ou menos,Indígena,"3015,5"
3,1999,1º grau completo ou menos,Não informado,"3254,2"
4,1999,1º grau completo ou menos,Negra,"3312,3"
...,...,...,...,...
325,2020,Superior completo ou mais,Amarela,"9471,4"
326,2020,Superior completo ou mais,Branca,"9089,9"
327,2020,Superior completo ou mais,Indígena,"7412,6"
328,2020,Superior completo ou mais,Não informado,"9111,1"


In [7]:
df.rename(columns={'escolaridade_formacao':'grau_instrucao','raca':'cor_raca', 'liquido':'remuneracao_liquida'}, inplace=True) #padronizando
df


,ano,grau_instrucao,cor_raca,remuneracao_liquida
0,1999,1º grau completo ou menos,Amarela,"3320,7"
1,1999,1º grau completo ou menos,Branca,"3543,9"
2,1999,1º grau completo ou menos,Indígena,"3015,5"
3,1999,1º grau completo ou menos,Não informado,"3254,2"
4,1999,1º grau completo ou menos,Negra,"3312,3"
...,...,...,...,...
325,2020,Superior completo ou mais,Amarela,"9471,4"
326,2020,Superior completo ou mais,Branca,"9089,9"
327,2020,Superior completo ou mais,Indígena,"7412,6"
328,2020,Superior completo ou mais,Não informado,"9111,1"


In [8]:
df.dtypes

ano                     int64
grau_instrucao         object
cor_raca               object
remuneracao_liquida    object
dtype: object

In [9]:
df['remuneracao_liquida'] = df['remuneracao_liquida'].str.replace(',','.').astype(float)
df

,ano,grau_instrucao,cor_raca,remuneracao_liquida
0,1999,1º grau completo ou menos,Amarela,3320.7
1,1999,1º grau completo ou menos,Branca,3543.9
2,1999,1º grau completo ou menos,Indígena,3015.5
3,1999,1º grau completo ou menos,Não informado,3254.2
4,1999,1º grau completo ou menos,Negra,3312.3
...,...,...,...,...
325,2020,Superior completo ou mais,Amarela,9471.4
326,2020,Superior completo ou mais,Branca,9089.9
327,2020,Superior completo ou mais,Indígena,7412.6
328,2020,Superior completo ou mais,Não informado,9111.1


Subindo para o GBQ

In [10]:
client = bigquery.Client()
dataset_ref = client.dataset('perfil_remuneracao')

c:\Users\ana.sales_republica\Anaconda\lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [11]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referência da observação'), 
 bigquery.SchemaField('grau_instrucao','STRING',description='Escolaridade da pessoa ou do vínculo observado'),
 bigquery.SchemaField('cor_raca','STRING',description='Raça/cor da pessoa observada'),
 bigquery.SchemaField('remuneracao_liquida','FLOAT',description='Remuneração liquida da pessoa observada')]

In [12]:
table_ref = dataset_ref.table('SIAPE_remuneracao_liquida_executivo_raca_escolaridade') # nome da tabela no padrão FONTE_algo_intuitivo_dado
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=9da0e495-79cd-4a21-a4f8-7eae537305e0>